# Part 3: Classification (40 Points) In this problem, you will develop a model to predict whether income exceeds $50K/yr based on census data.2


In [42]:
import numpy as np
import pandas as pd
import patsy as pt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy as pt
#from pyearth import Earth #conda install sklearn-contrib-py-earth

from IPython.display import HTML
from ipywidgets import interact
import ipywidgets as widgets
import copy
import warnings
warnings.filterwarnings('ignore')

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
#import graphviz 
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model as lm
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC, SVC
#from mlxtend.plotting import plot_decision_regions
# Import classes from scikit-learn for logistic regression, LDA, QDA, and KNN classification
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif

%matplotlib inline

### a) Use the code in Blackboard to create the adult data set.


In [43]:
from ucimlrepo import fetch_ucirepo
adult = fetch_ucirepo(id=2)

X = adult.data.features
y = adult.data.targets

#print(adult.metadata)

#print(adults.variables)

### b) Explore the data graphically in order to investigate the association between income and the other features. Which of the other features seem most likely to be useful in predicting income? Scatterplots and boxplots may be useful tools to answer this question. Describe your findings.


In [44]:
X['marital-status'].value_counts()

Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: marital-status, dtype: int64

In [59]:
y['income'] = y['income'].replace('>50K.', '>50K').replace('<=50K.', '<=50K')


In [60]:
X = pd.get_dummies(X, drop_first=True)
X.shape

(48842, 100)

In [61]:
# don't want to write out the equation!
f = 'income ~ 1 +' + ' + '.join(X.columns.drop(['fnlwgt','education-num']))

### c) Split the data into an 80% training set and a 20% test set. Set the seed at 1303.


In [62]:
import random
random.seed(10)
train = np.random.random(len(y)) >.2
train.sum() / train.shape[0]

0.8001924573113304

### d) Perform LDA on the training data in order to predict income using the variables that seemed most associated with income in (b). What is the test error of the model obtained?


In [63]:
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X[train], y[train])

LinearDiscriminantAnalysis()

In [64]:
lda_pred = lda_clf.predict(X[~train])

In [65]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
conf_matrix = confusion_matrix(y[~train], lda_pred)
conf_matrix

array([[6919,  517],
       [ 963, 1360]], dtype=int64)

In [66]:
pd.DataFrame(
    confusion_matrix(y[~train],lda_pred),
    columns=['<50k','>50k'],
    index=['<50k','>50k']
)

,<50k,>50k
<50k,6919,517
>50k,963,1360


### e) Perform QDA on the training data in order to predict income using the variables that seemed most associated with income in (b). What is the test error of the model obtained?


In [67]:
qda_clf = QuadraticDiscriminantAnalysis()
qda_clf.fit(X[train],y[train])

QuadraticDiscriminantAnalysis()

In [71]:
qda_pred = qda_clf.predict(X[~train])
pd.DataFrame(
    confusion_matrix(
        y[~train], qda_pred),
    columns=['<50k','>50k'],
    index=['<50k','>50k']
)

,<50k,>50k
<50k,1009,6427
>50k,74,2249


### f) Perform logistic regression on the training data in order to predict income using the variables that seemed most associated with income in (b). What is the test error of the model obtained?


In [74]:
log_reg = LogisticRegression(
    penalty = None,
    solver = 'lbfgs'
)
log_reg.fit(
    X[train],
    y[train]
)

LogisticRegression(penalty=None)

In [78]:
log_reg_pred = log_reg.predict(X[~train])

In [79]:
pd.DataFrame(
    confusion_matrix(
        y[~train], log_reg_pred),
    columns=['<50k','>50k'],
    index=['<50k','>50k']
)

,<50k,>50k
<50k,7217,219
>50k,1726,597


### g) Perform KNN on the training data, with several values of K, in order to predict income. Use only the variables that seemed most associated with income in (b). What test errors do you obtain? Which value of K seems to perform the best on this data set?


In [86]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Create a KNeighborsClassifier instance
knn = KNeighborsClassifier()

# Define parameter grid for hyperparameter tuning
param_grid = {'n_neighbors': np.arange(1, 5)}

# Initialize GridSearchCV
knn_gscv = GridSearchCV(knn, param_grid, cv=5)

# Train the model using the training data
# Note: Assuming `train` is a boolean mask indicating training samples
# Make sure to replace it with your actual train-test split logic if needed
knn_gscv.fit(X[train], y[train])


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([1, 2, 3, 4])})

### j) Choose which model predicts income the best and justify your choice.

In [87]:
knn_gscv.best_estimator_
knn_pred = knn_gscv.best_estimator_.predict(X[~train])

In [88]:
pd.DataFrame(
    confusion_matrix(
        y[~train], knn_pred),
    columns=['<50k','>50k'],
    index=['<50k','>50k']
)

,<50k,>50k
<50k,7106,330
>50k,1686,637
